In [1]:
from datetime import datetime, timedelta, date, time
import random
from typing import List, Dict, Optional
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
import json
import google.generativeai as genai
from google.colab import userdata

# --- Global Data Definitions ---
DATASET = [
    {'id': 1, 'nome': 'Salada de Quinoa', 'calorias': 220, 'preco': 18.0, 'saudavel': True},
    {'id': 2, 'nome': 'Peito de Frango Grelhado + Arroz Integral', 'calorias': 380, 'preco': 25.0, 'saudavel': True},
    {'id': 3, 'nome': 'Smoothie Proteico', 'calorias': 300, 'preco': 15.0, 'saudavel': True},
    {'id': 4, 'nome': 'Omelete Vegetal', 'calorias': 240, 'preco': 12.0, 'saudavel': True},
    {'id': 5, 'nome': 'Bowl de Açaí Light', 'calorias': 260, 'preco': 20.0, 'saudavel': True},
    {'id': 6, 'nome': 'Hambúrguer Tradicional', 'calorias': 850, 'preco': 30.0, 'saudavel': False},
    {'id': 7, 'nome': 'Pizza Portuguesa (fatia)', 'calorias': 650, 'preco': 22.0, 'saudavel': False},
    {'id': 8, 'nome': 'Doce de Chocolate', 'calorias': 420, 'preco': 8.0, 'saudavel': False},
    {'id': 9, 'nome': 'Wrap de Atum', 'calorias': 320, 'preco': 17.0, 'saudavel': True},
    {'id': 10,'nome': 'Panqueca Proteica', 'calorias': 360, 'preco': 14.0, 'saudavel': True},
]

PLANS = {'3m': 90, '6m': 180, '12m': 365}

# --- IMC Calculation and Classification Functions ---
def calcular_imc(peso: float, altura: float) -> float:
    return peso / (altura ** 2)

def classificar_imc(imc: float) -> str:
    if imc < 18.5:
        return "Está abaixo do peso"
    elif 18.5 <= imc < 25:
        return "O peso está normal"
    elif 25 <= imc < 30:
        return "Está com sobrepeso"
    elif 30 <= imc < 35:
        return "Obesidade de grau 1"
    elif 35 <= imc < 40:
        return "Obesidade de grau 2"
    else:
        return "Obesidade de grau 3"

# --- Google Gemini AI Setup ---
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)
gemini_model = genai.GenerativeModel('gemini-pro-latest')

def generate_healthy_meal_suggestion(user_imc: float, existing_dataset: List[Dict]) -> str:
    imc_category = classificar_imc(user_imc)
    dataset_str = "\n".join([f"- {item['nome']} ({item['calorias']} kcal, saudavel: {item['saudavel']})" for item in existing_dataset])
    prompt = f"""
    Considerando que o usuário tem um IMC '{imc_category}',
    e o seguinte conjunto de dados de refeições:
    {dataset_str}

    Sugira uma nova refeição saudável e equilibrada, que não esteja no dataset, com foco em:
    - Ser criativa e saborosa.
    - Apropriada para essa classificação de IMC.
    - Incluir ingredientes principais e calorias estimadas.
    - Não incluir preço. Seja conciso.
    """
    try:
        response = gemini_model.generate_content(prompt)
        return response.text
    except Exception as e:
        return f"Desculpe, não consegui gerar uma sugestão de refeição saudável no momento. Erro: {e}"

# --- User Class Definition ---
class User:
    def __init__(self, user_id: str, imc: float):
        self.user_id = user_id
        self.imc = imc
        self.plan = None
        self.schedule_time: Optional[time] = None
        self.consumption_log: Dict[str, bool] = {}
        self.rewards_given: int = 0
        self.current_consecutive_healthy_streak: int = 0
        self.reward_options: List[Dict] = []
        self.healthy_streak_goal = None
        self.last_goal_reset_date: Optional[date] = None

    def subscribe(self, plan_key: str, start_date: date = None):
        if start_date is None:
            start_date = date.today()
        days = PLANS[plan_key]
        end_date = start_date + timedelta(days=days)
        self.plan = (plan_key, start_date.isoformat(), end_date.isoformat())
        print(f"[Plano] Assinatura {plan_key} válida até {end_date}")

    def set_schedule_time(self, hora: int, minuto: int = 0):
        self.schedule_time = time(hora, minuto)
        print(f"[Schedule] Horário fixo definido para {self.schedule_time}")

    def set_healthy_streak_goal(self, goal: int, current_date: date = None):
      if goal > 0:
        self.healthy_streak_goal = goal
        self.current_consecutive_healthy_streak = 0
        self.rewards_given = 0
        self.reward_options = []
        self.last_goal_reset_date = current_date if current_date else date.today()
        print(f"[Meta] Nova meta definida para {self.healthy_streak_goal} dias consecutivos. Contagem reiniciada!")
        print(f"[Meta] Total de dias saudáveis registrados até agora: {self.total_healthy_days()} ")
      else:
        print("[Meta] A meta de dias saudáveis deve ser um número positivo.")

    def is_plan_active(self, on_date: date = None) -> bool:
        if self.plan is None:
            return False
        if on_date is None:
            on_date = date.today()
        _, _, end_iso = self.plan
        end = datetime.fromisoformat(end_iso).date()
        return on_date <= end

    def log_consumption(self, day: date, consumed: bool):
        self.consumption_log[day.isoformat()] = consumed

        # Update streak
        self.current_consecutive_healthy_streak = 0
        if consumed:
            self.current_consecutive_healthy_streak = 1
            previous_date = day - timedelta(days=1)
            while previous_date.isoformat() in self.consumption_log and self.consumption_log[previous_date.isoformat()]:
                # Ensure date is after last_goal_reset_date if set
                if self.last_goal_reset_date and previous_date < self.last_goal_reset_date:
                    break
                self.current_consecutive_healthy_streak += 1
                previous_date -= timedelta(days=1)

            # Incentivo estilo C
            if self.healthy_streak_goal is not None and self.current_consecutive_healthy_streak == self.healthy_streak_goal:
                print(f"🎉 Uhuu! Você completou {self.healthy_streak_goal} dias saudáveis seguidos! 💪🏽 Hora de resgatar sua recompensa!")

        # Mensagem diária
        print(f"✅ Refeição registrada! Streak atual: {self.current_consecutive_healthy_streak} dias consecutivos de hábitos saudáveis! Continue assim! 💥")
        print(f"Total de dias saudáveis registrados: {self.total_healthy_days()}")

    def total_healthy_days(self) -> int:
        return sum(1 for v in self.consumption_log.values() if v)

    def _calculate_consecutive_healthy_streaks(self) -> int:
        # Filter consumption log based on last_goal_reset_date
        filtered_consumption = {
            d_str: consumed for d_str, consumed in self.consumption_log.items()
            if not self.last_goal_reset_date or date.fromisoformat(d_str) >= self.last_goal_reset_date
        }

        sorted_dates = sorted(filtered_consumption.keys())
        streaks_completed = 0
        consecutive_healthy_days = 0

        if not sorted_dates:
            return 0

        date_objects = [date.fromisoformat(d) for d in sorted_dates]

        for i in range(len(date_objects)):
            current_date = date_objects[i]
            is_healthy = filtered_consumption[current_date.isoformat()]

            if is_healthy:
                if i == 0 or (current_date - date_objects[i-1]).days == 1:
                    consecutive_healthy_days += 1
                else:
                    consecutive_healthy_days = 1
            else:
                consecutive_healthy_days = 0

            if self.healthy_streak_goal is not None and consecutive_healthy_days >= self.healthy_streak_goal:
                streaks_completed += 1
                consecutive_healthy_days = 0

        return streaks_completed

    def _generate_reward_options(self):
        non_healthy_items = [p for p in DATASET if not p['saudavel']]
        num_options = min(len(non_healthy_items), 3)

        if not non_healthy_items:
            print("[Reward] Não há itens não saudáveis disponíveis para recompensas.")
            self.reward_options = []
            return

        selected_items_for_ai = random.sample(non_healthy_items, num_options)
        prompt = f"""
        Gere até 3 opções de recompensa com base nos seguintes itens não saudáveis.
        Para cada item, aplique um desconto aleatório entre 10% e 50% no preço original (`preco`)
        e forneça uma descrição curta e atraente (`descricao_ai`).
        O retorno DEVE ser APENAS um array JSON.
        Itens disponíveis (nome e preço original):
        {json.dumps([{'nome': item['nome'], 'preco': item['preco']} for item in selected_items_for_ai])}
        """

        try:
            response = gemini_model.generate_content(prompt)
            raw_ai_response_text = response.text

            if raw_ai_response_text.strip().startswith('```json') and raw_ai_response_text.strip().endswith('```'):
                json_string = raw_ai_response_text.strip()[len('```json'):-len('```')].strip()
            else:
                json_string = raw_ai_response_text.strip()

            ai_rewards = json.loads(json_string)
            self.reward_options = []

            for item in ai_rewards:
                original_item = next((d for d in DATASET if d['nome'] == item.get('nome')), None)
                if original_item:
                    original_price = original_item['preco']
                    ai_discounted_price = item.get('preco_com_desconto')
                    ai_description = item.get('descricao_ai', 'Desfrute de um mimo especial!')

                    if ai_discounted_price is None or not isinstance(ai_discounted_price, (int, float)) or ai_discounted_price >= original_price:
                        discount_percentage = random.uniform(0.10, 0.50)
                        calculated_discounted_price = round(original_price * (1 - discount_percentage), 2)
                    else:
                        calculated_discounted_price = round(ai_discounted_price, 2)

                    self.reward_options.append({
                        'nome': original_item['nome'],
                        'preco': original_price,
                        'preco_com_desconto': calculated_discounted_price,
                        'descricao_ai': ai_description
                    })

            print(f"[Reward] Geradas {len(self.reward_options)} opções de recompensa com IA.")
        except Exception as e:
            print(f"[Reward] Erro ao gerar opções de recompensa com IA: {e}")
            self.reward_options = []
            for item in selected_items_for_ai:
                item_copy = item.copy()
                if 'preco' in item_copy:
                    item_copy['preco_com_desconto'] = round(item_copy['preco'] * random.uniform(0.5, 0.9), 2)
                else:
                    item_copy['preco_com_desconto'] = item_copy.get('preco', 0)
                item_copy['descricao_ai'] = "Desfrute de um mimo especial!"
                self.reward_options.append(item_copy)
            print(f"[Reward] Geradas {len(self.reward_options)} opções de recompensa (fallback).")

    def available_rewards(self) -> int:
        total_streaks = self._calculate_consecutive_healthy_streaks()
        rewards_count = max(0, total_streaks - self.rewards_given)
        if rewards_count > 0 and not self.reward_options:
            self._generate_reward_options()
        return rewards_count

    def redeem_reward(self, selected_item: Dict) -> Optional[Dict]:
        if self.available_rewards() <= 0:
            print("😅 Nenhuma recompensa disponível para resgatar.")
            return None
        item = selected_item
        self.reward_options = []
        self.rewards_given += 1
        print(f"🎁 Parabéns! Você resgatou: {item['nome']} por R${item['preco_com_desconto']:.2f}. Aproveite! 😋")
        return item

# --- ML Recommendation Model Training Function ---
def train_recommendation_model(user_history: Dict[str, bool], dataset: List[Dict], imc: float):
    rows = []
    for day, consumed in user_history.items():
        for p in dataset:
            row = {
                'imc': imc,
                'calorias': p['calorias'],
                'saudavel': int(p['saudavel']),
                'dia': day,
                'consumido': int(consumed)
            }
            rows.append(row)

    if not rows:
        def default_recommend_with_ml(top_n=5):
            df_default = pd.DataFrame(dataset)
            df_default['score'] = df_default.apply(lambda x: (2 if x['saudavel'] else 1) - x['calorias'] / 1000, axis=1)
            return df_default.head(top_n).to_dict(orient='records')
        return default_recommend_with_ml

    df = pd.DataFrame(rows)
    X = df[['imc', 'calorias', 'saudavel']]
    y = df['consumido']
    model = RandomForestClassifier(n_estimators=50, random_state=42)
    model.fit(X, y)

    def recommend_with_ml(top_n=5):
        df_pred = pd.DataFrame(dataset)
        df_pred['saudavel'] = df_pred['saudavel'].astype(int)
        X_pred = df_pred[['calorias', 'saudavel']].copy()
        X_pred['imc'] = imc
        X_pred = X_pred[['imc', 'calorias', 'saudavel']]
        df_pred['score'] = model.predict_proba(X_pred)[:,1]
        df_pred = df_pred.sort_values('score', ascending=False)
        return df_pred.head(top_n).to_dict(orient='records')

    return recommend_with_ml

# --- Main Application Function ---
def main():
    print("=== Bem-vindo ao Ifood Fit ===")
    user_id = input("Digite seu nome: ")
    peso = float(input("Digite seu peso (kg): "))
    altura = float(input("Digite sua altura (m, ex: 1.65): "))

    imc = calcular_imc(peso, altura)
    user = User(user_id=user_id, imc=imc)
    print(f"📊 Seu IMC: {imc:.1f} ({classificar_imc(imc)})")

    print("Planos disponíveis: 3m, 6m, 12m")
    plano = input("Escolha um plano: ")
    user.subscribe(plano)

    hora = int(input("Defina a hora fixa para receber sua refeição (0-23): "))
    minuto = int(input("Defina o minuto (0-59): "))
    user.set_schedule_time(hora, minuto)

    valid_goals = [3, 6, 12, 20, 30]
    current_simulated_date = date.today()
    while True:
        try:
            goal_input = int(input(f"Defina sua meta de dias saudáveis consecutivos {valid_goals}: "))
            if goal_input in valid_goals:
                user.set_healthy_streak_goal(goal_input, current_simulated_date)
                break
            else:
                print("Meta inválida. Escolha entre 3, 6, 12, 20 ou 30 dias.")
        except ValueError:
            print("Entrada inválida. Digite um número.")

    ml_recommender = train_recommendation_model(user.consumption_log, DATASET, user.imc)


    while True:
        print("\n=== 🚀 Menu Ifood Fit 🚀 ===")
        print("1 - Registrar refeição saudável")
        print("2 - Resgatar sua recompensa")
        print("3 - Quero sugestão de refeição fitness")
        print("4 - Redefinir meta de dias saudáveis")
        print("0 - Sair do app")
        escolha = input("Escolha uma opção: ")

        if escolha == "1":
            confirm = input("Você consumiu a refeição saudável hoje? (s/n): ").lower() == 's'
            user.log_consumption(current_simulated_date, confirm)
            current_simulated_date += timedelta(days=1)
        elif escolha == "2":
            rewards_count = user.available_rewards()
            if rewards_count <= 0:
                print("😅 Nenhuma recompensa disponível no momento. Continue firme na sua streak!")
                continue
            print(f"Você tem {rewards_count} recompensa(s) disponível(eis):")
            for i, item in enumerate(user.reward_options):
                print(f"  {i+1}. {item['nome']} (R${item['preco_com_desconto']:.2f}) - {item.get('descricao_ai', 'Desfrute de um mimo especial!')}")
            while True:
                try:
                    choice = int(input("Escolha o número da recompensa que deseja resgatar: "))
                    if 1 <= choice <= len(user.reward_options):
                        selected_item = user.reward_options[choice - 1]
                        user.redeem_reward(selected_item)
                        break
                    else:
                        print("Escolha inválida. Digite um número da lista.")
                except ValueError:
                    print("Entrada inválida. Digite um número.")
        elif escolha == "3":
            print("🤖 IA está pensando na refeição perfeita pra você... 🍽️")
            suggestion = generate_healthy_meal_suggestion(user.imc, DATASET)
            print("\nSugestão da IA:")
            print(suggestion)

        elif escolha == "4":
            print(f"Meta atual: {user.healthy_streak_goal} dias consecutivos")
            while True:
                try:
                    goal_input = int(input(f"Escolha a nova meta de dias saudáveis consecutivos {valid_goals}: "))
                    if goal_input in valid_goals:
                        user.set_healthy_streak_goal(goal_input, current_simulated_date)
                        break
                    else:
                        print(f"Meta inválida. Escolha entre {valid_goals}.")
                except ValueError:
                    print("Entrada inválida. Digite um número.")

        elif escolha == "0":
            print("Saindo do app... Até mais! 👋")
            break
        else:
            print("Opção inválida, tente novamente.")

if __name__ == "__main__":
    main()


=== Bem-vindo ao Ifood Fit ===
Digite seu nome: Henrique
Digite seu peso (kg): 85
Digite sua altura (m, ex: 1.65): 1.80
📊 Seu IMC: 26.2 (Está com sobrepeso)
Planos disponíveis: 3m, 6m, 12m
Escolha um plano: 3m
[Plano] Assinatura 3m válida até 2026-02-21
Defina a hora fixa para receber sua refeição (0-23): 16
Defina o minuto (0-59): 30
[Schedule] Horário fixo definido para 16:30:00
Defina sua meta de dias saudáveis consecutivos [3, 6, 12, 20, 30]: 3
[Meta] Nova meta definida para 3 dias consecutivos. Contagem reiniciada!
[Meta] Total de dias saudáveis registrados até agora: 0 

=== 🚀 Menu Ifood Fit 🚀 ===
1 - Registrar refeição saudável
2 - Resgatar sua recompensa
3 - Quero sugestão de refeição fitness
4 - Redefinir meta de dias saudáveis
0 - Sair do app
Escolha uma opção: 4
Meta atual: 3 dias consecutivos
Escolha a nova meta de dias saudáveis consecutivos [3, 6, 12, 20, 30]: 3
[Meta] Nova meta definida para 3 dias consecutivos. Contagem reiniciada!
[Meta] Total de dias saudáveis registr